# Python Generators

## Functions that behave like iterators 

Functions that keep on giving. Use them in `for` loops.

In [ ]:
def function():
    """A standard function."""
    return [42]

function()

In [ ]:
def generator():
    """Including a `yield` statement makes a generator"""
    yield 42
    
generator()

In [ ]:
for i in function():
    print('f', i)

In [ ]:
for i in generator():
    print('g', i)

## Generator expressions

Simple iteration without need to create intermediate lists.

In [ ]:
# List comprehension
[x for x in range(10) if x % 2]

In [ ]:
# Generator expression
(x for x in range(10) if x % 2)

Use it in place as an argument:

In [ ]:
import random
set(random.random() for _ in range(5))

## Lazy evaluation

Functions that keep on living. Code only runs when it has to.

In [ ]:
def generator():
    print("Hi!")
    yield 42
    print("Done!")

In [ ]:
g = generator()  # Output?

In [ ]:
list(generator())  # Output?

In [ ]:
g = generator()

In [ ]:
next(g)

In [ ]:
def fibo():
    a = 0
    yield a
    b = 1
    yield b
    while True:  # endless loop!
        c = a + b
        yield c
        a, b = b, c
        

In [ ]:
from itertools import takewhile  # itertools has handy tools for dealing with generators
list(takewhile(lambda x: x<10, fibo()))

## What for?

- stream processing, consumer pulls
  - reading HTTP body that can arrive in chunks
  - database result sets
  
- don't need or want to keep all data in memory
  - process gigantic CSV file
  
- endless results / unknown how many needed
    - counter
    
- building block for context managers...

- coroutines for async processing...


## Gotchas

### Usable only once

In [ ]:
g = (c for c in 'Hello World!' if c.isupper())
print(list(g))

In [ ]:
print(list(g))

### Cleanup non-deterministic

In [ ]:
def read_lines(filename):
    try:
        with open(filename) as f:
            print('--- file opened')
            for line in f:
                yield line.rstrip()  # remove trailing whitespace
    finally:
        print('--- file closed')

In [ ]:
reader = read_lines('Python Generators.ipynb')
for i, l in enumerate(reader):
    if i > 5:
        break
    print(i, l)

In [ ]:
del reader

In [ ]:
from contextlib import closing
with closing(read_lines('Python Generators.ipynb')) as reader:
    print(next(reader))

In [ ]:
# Beware exceptions that get raised on cleanup

Also beware: https://amir.rachum.com/blog/2017/03/03/generator-cleanup/